# ParFuMor (version YAML & Objets)

- Ajout d'un %store pour la gestion des numéros de kanoniks (16/04/20)

- Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [182]:
# -*- coding: utf8 -*-
import os
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
#import cellbell 

In [183]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

# Gestion partagée des numéros à traiter

le block suivant permet de partager les numéros à traiter entre les différents Notebooks.
- %store -r variable lit la variable dans le stock
- %store variable stocke la variable

In [184]:
%store -r numerosKalaba typeKalaba
numerosKalaba=[1,2,3,4,5]
numerosKalaba=[2]
print numerosKalaba
typeKalaba="Kanonik"
%store numerosKalaba 
%store typeKalaba

[2]
Stored 'numerosKalaba' (list)
Stored 'typeKalaba' (str)


In [185]:
home = expanduser("~")

if typeKalaba!="Kanonik":
    repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
    serie=repertoire+"21-"
    nomsKalabas=[serie+"K%d/"%num for num in numerosKalaba]
else:
    repertoire=home+"/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/"
    serie=repertoire+"K-22/"
    nomsKalabas=[serie+"Kanonik-%02d/"%num for num in numerosKalaba]

In [186]:
def getExemple(lDict):
    exKey=lDict.keys()[0]
    if isinstance(lDict[exKey],dict):
        result=getExemple(lDict[exKey])
    elif isinstance(lDict[exKey],list):
        result=lDict[exKey][0]
    return result

In [187]:
PFM.duplicateErrors=[]
for serie in nomsKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.safe_load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.safe_load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    lexiqueTestPrep=getExemple(stems["PREP"])
    lexiqueTestNoun=getExemple(stems["NOM"])
    lexiqueTestHyper=getExemple(stems["ADJ"])
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.safe_load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.safe_load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.safe_load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
    [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/K-22/Kanonik-02/
NOM : Genre Nombre
VER : Temps Pers
ADJ : Genre Nombre
PRO : Genre Nombre
DET : Genre Nombre
head stems
head stems,NOM
head stems,NOM,N1
head stems,NOM,N2
head stems,DET


In [188]:
stems["PREP"]

{'Zik': ['sur'],
 'akad': ['pendant'],
 'beN': ['entre'],
 'fu': ['vers'],
 'gopun': [u'apr\xe8s'],
 'lal': ['pour'],
 'mileb': ['avec'],
 'ne': ['en'],
 'ni': [u'\xe0'],
 'po': ['dans'],
 'toS': ['de']}

In [189]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
print PFM.lexique.formeLexeme[lexiqueTestNoun][0]
print PFM.lexique.formeLexeme[lexiqueTestPrep][0]

nouveau
morceau.N1.F
pour


In [190]:
paradigmes.getSigmas(["VER"])

[u'VER, Temps=PRS, Pers=3Sg',
 u'VER, Temps=PRS, Pers=3Pau',
 u'VER, Temps=PRS, Pers=3Pl',
 u'VER, Temps=PST, Pers=3Sg',
 u'VER, Temps=PST, Pers=3Pau',
 u'VER, Temps=PST, Pers=3Pl']

In [191]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [192]:
regles.getRules("VER",'CF=V1, Temps=PRS, Pers=3PL')
# regles.getRules("PRO","Nombre=Du, Genre=C")

[('X+a', 'Temps=PRS')]

In [193]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{u'F': {('Sonizi', u'fl\xe8che'),
  ('bagape', u'arbre'),
  ('bizili', u'jumelle'),
  ('bokenuN', u'aventure'),
  ('buriza', u'livre'),
  ('dasetu', u'l\xe9gende'),
  ('fevegi', u'feu'),
  ('fivuS', u'd\xe9sert'),
  ('geSiso', u'histoire'),
  ('kaSeZu', u'pause'),
  ('kurige', u'guerri\xe8re'),
  ('mopinu', u'tr\xe9sor'),
  ('razebu', u'voyage'),
  ('segika', u'dessin'),
  ('sekuZi', u'repas'),
  ('tapilu', u'morceau'),
  ('telofi', u'tellaufi'),
  ('tizer', u'cit\xe9'),
  ('voseda', u'grotte')},
 u'M': {('Sigolo', u'parchemin'),
  ('Zezame', u'nom'),
  ('arutur', u'Arthur'),
  ('bodola', u'fl\xfbte'),
  ('dezato', u'village'),
  ('faduNe', u'pi\xe8ge'),
  ('firule', u'lunette'),
  ('gurusu', u'bouche'),
  ('lubrik', u'chef'),
  ('muSula', u'dragon'),
  ('paterik', u'Patrick'),
  ('rorapo', u'dinosaure'),
  ('rutoza', u'pomme'),
  ('satame', u'tronc'),
  ('tomoda', u'amiti\xe9'),
  ('vegalum', u'chemin'),
  ('vosegaN', u'entr\xe9e'),
  ('vozubli', u'gar\xe7on'),
  ('zaNir', u'Zagnir')}

In [194]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

désert.N1.F fivuS
reconnaître bolat
entrée.N1.M vosegaN
disparaître turbul
enflammé potika
parchemin.N1.M Sigolo
livre.N2.F buriza
Zagnir.N2.M zaNir
ouvrir tubop
flèche.N2.F Sonizi
force.N1.N Seteruk
mettre taZm
rencontrer mobid
vers fu
montrer elets
de toS
pomme.N2.M rutoza
trouver mimuz
IND.ART Ze
perdu ferla
tronc.N2.M satame
voir iSdif
nouveau niNu
piège.N1.M faduNe
reposer bukr
partir gotop
morceau.N1.F tapilu
ancien moldu
voler flup
histoire.N1.F geSiso
en ne
souffler venom
dans po
bouche.N2.M gurusu
revoir fidj
chef.N2.M lubrik
raconter kaNil
chemin.N1.M vegalum
englouti glabo
dévoiler magob
sur Zik
quête.N1.N savomu
sauveur.N2.N leNafe
féliciter Zizel
pendant akad
DEF.ART nu
village.N1.M dezato
tellaufi.N1.F telofi
à ni
être vokel
lunette.N2.M firule
voyage.N1.F razebu
pause.N1.F kaSeZu
lutin.N2.N lepuruSo
aventure.N1.F bokenuN
sept geZo
trésor.N2.F mopinu
fond.N1.N kagumi
détruire Zupet
garçon.N1.M vozubli
recherche.N1.N foreSu
cité.N1.F tizer
bloquer kolb
troupe.N2.N kunati
a

In [195]:
ding()